In [4]:
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import random

In [2]:
url = "https://olympics.com/tokyo-2020/olympic-games/en/results/all-sports/athletes.htm"

In [3]:
base_url = "https://olympics.com/tokyo-2020/olympic-games/"

# Using selenium to open chrome browser

In [37]:
driver_path = "chromedriver//chromedriver"

In [3]:
wd = webdriver.Chrome(executable_path=driver_path)

In [6]:
wd.get(url)

In [7]:
html = wd.page_source

In [8]:
soup = bs(html)

# Setting up proxy with selenium

In [38]:
from selenium import webdriver
PROXY = "207.244.227.169:443"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)
chrome = webdriver.Chrome(options=chrome_options,executable_path=driver_path)
chrome.get("https://www.olympics.com")

# Looping through pages to grab athlete links

In [12]:
links = []
for i in range(584):
    links.extend(soup.select("div.playerTag a"))
    wd.find_element_by_id('entries-table_next').click()
    time.sleep(random.uniform(5, 10))
    html = wd.page_source
    soup = bs(html)

In [ ]:
link_list = []

In [ ]:
for link in links:
    link = link["href"]
    link_list.append(link[9:])

In [ ]:
link_list

# Save/load json links

In [16]:
import json
def save_data(title,data):
    with open(title,'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [23]:
save_data('data//athlete_link_list.json', link_list)

In [31]:
import json
def load_data(title):
    with open(title, encoding='utf-8') as f:
        return json.load(f)

In [32]:
loaded_links = load_data('data//athlete_link_list.json')

In [41]:
loaded_links[3880]

'en/results/equestrian/athlete-profile-n1482494-hanna-mary.htm'

# Looping through pages and saving html pages

In [ ]:
for i in range(len(loaded_links)):
    with open(f"html_scripts//{i}.html", "w", encoding="utf-8") as file:
        wd.get(base_url + loaded_links[i])
        if i % 20 == 0 and i != 0:
            time.sleep(3)
        html = bs(wd.page_source)
        test_soup = html.select('main.wrs-content')
        file.write(str(test_soup))

# Looping through files in directory then calling scraping function

In [99]:
import os
def load_pages():
    i = 1
    directory ='C://Users//danag//Desktop//Coding Stuff//learnpandas//html_scripts'
    for filename in os.listdir(directory):
        if filename.endswith('.html'):
            fname = os.path.join(directory,filename)
            with open(fname, 'r', encoding="utf-8") as f:
                soup = bs(f)
                try:
                    scrape_html(i, soup)
                    i = i + 1
                    print(i)
                except Exception as e:
                    print(e)

# Scraping data from html

In [98]:
df = pd.DataFrame(columns=["Name", "Sex", "Age", "Team", "NOC", "Year", "Season", "City", "Sport", "Event", "Medal"])
def scrape_html(i, soup):
    #Columns for athlete dataframe
    columns=["Name", "Sex", "Age", "Team", "NOC", "Year", "Season", "City", "Sport", "Event", "Medal"]
    #Creating the row of data for temporary df
    athlete_info = []
    #Collecting data from html page
    name = soup.find("div", class_="col-lg-9").h1.string.strip()
    noc = soup.select("div.col-9 div.playerTag")[0]['country']
    team = soup.select("div.col-9 div.playerTag a")[0].text
    age = soup.select("div.col-9 div.col-md-6")[0].text.strip('\n').split("\n\n")[1].split('\n')[1]
    sex = soup.select("div.col-9 div.col-md-6")[0].text.strip('\n').split("\n\n")[2].split('\n')[1]
    year = '2020'
    season = 'Summer'
    city = 'Tokyo'
    sport = soup.select("table.table-schedule td")[0].text.strip()
    events_table = soup.find("table", class_='table-schedule').find("tbody").find_all('tr')
    #Storing list of all events and medals won
    event_list = []
    medal_list = []
    #looping through events in events table
    for event in events_table:
        event_list.append(event.find_all('td')[1].find('a').text)
    medals_table = soup.find("table", class_='table-schedule').find("tbody").find_all('tr')
    #looping throud medals in medals table
    for medal in medals_table:
        if len(medal.find_all('td')[-1].find_all()) == 0:
            medal_list.append('N/A')
        else:
            medal = medal.find_all('td')[-1].find('img')['title'].split(' ')[0]
            medal_list.append(medal)
    #Appending all data of athlete to table above
    athlete_info.append([name, sex, age, team, noc, year, season, city, sport, event_list, medal_list])
    #Creating dataframe of with athlete data to columns above
    athlete_df = pd.DataFrame(athlete_info, columns = columns)
    global df
    #Appending athlete dataframe to ain dataframe
    df = df.append(athlete_df)

In [ ]:
df

# Saving original data frame

In [105]:
df.to_csv('archive//unexploded_olympics.csv', index=False)

# Saving exploded data frame

In [102]:
# exploding lists in columns of "Event" and "Medal" 
final_df = df.explode(['Event', 'Medal'])

In [103]:
final_df.to_csv('archive//exploded_olympics.csv', index=False)

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal
0,RAHAYU Apriyani,Female,23,Indonesia,INA,2020,Summer,Tokyo,Badminton,Women's Doubles,Gold
0,AALERUD Katrine,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Road Race,N/A
0,AALERUD Katrine,Female,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Individual Time Trial,N/A
0,ABBASALI Hamideh,Female,31,Islamic Republic of Iran,IRI,2020,Summer,Tokyo,Karate,Women's Kumite +61kg,N/A
0,AFONSO Salome,Female,23,Portugal,POR,2020,Summer,Tokyo,Athletics,Women's 1500m,N/A
...,...,...,...,...,...,...,...,...,...,...,...
0,TAN Qian Xiu Adele,Female,22,Singapore,SGP,2020,Summer,Tokyo,Shooting,10m Air Rifle Women,N/A
0,TAN Sze En,Female,20,Singapore,SGP,2020,Summer,Tokyo,Artistic Gymnastics,Women's Balance Beam,N/A
0,TAN Sze En,Female,20,Singapore,SGP,2020,Summer,Tokyo,Artistic Gymnastics,Women's Floor Exercise,N/A
0,TAN Ya-Ting,Female,27,Chinese Taipei,TPE,2020,Summer,Tokyo,Archery,Women's Individual,N/A


# Testing Exploding a dataframe

In [96]:
info_list = [['Daniel', 'Aguirre', "28", ['running', 'guitar', 'video games'],['1', '2', '3']]]

In [97]:
temp_df = pd.DataFrame(info_list, columns=['First Name', 'Last Name', 'Age', 'Hobbies', 'numbers'])

In [98]:
my_df = pd.DataFrame(columns=['First Name', 'Last Name', 'Age', 'Hobbies', 'numbers'])

In [99]:
temp_df

,First Name,Last Name,Age,Hobbies,numbers
0,Daniel,Aguirre,28,"[running, guitar, video games]","[1, 2, 3]"


In [100]:
my_df = my_df.append(temp_df)

In [101]:
my_df

,First Name,Last Name,Age,Hobbies,numbers
0,Daniel,Aguirre,28,"[running, guitar, video games]","[1, 2, 3]"


In [102]:
my_df.explode(['Hobbies', 'numbers'])

,First Name,Last Name,Age,Hobbies,numbers
0,Daniel,Aguirre,28,running,1
0,Daniel,Aguirre,28,guitar,2
0,Daniel,Aguirre,28,video games,3


# Cleaning data frame

In [20]:
olympics_df = pd.read_csv('archive//exploded_olympics.csv')

In [21]:
olympics_df['Name'] = olympics_df['Name'].str.title()

In [26]:
olympics_df['Name'] = pd.Series(olympics_df['Name'].str.split(' ').str[::-1].apply(lambda x : ' '.join(x)))

In [31]:
olympics_df['Sex'] = olympics_df['Sex'].str[0]

In [33]:
olympics_df.head()

,Name,Sex,Age,Team,NOC,Year,Season,City,Sport,Event,Medal
0,Katrine Aalerud,F,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Road Race,NaN
1,Katrine Aalerud,F,26,Norway,NOR,2020,Summer,Tokyo,Cycling Road,Women's Individual Time Trial,NaN
2,Nestor Abad,M,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's All-Around,NaN
3,Nestor Abad,M,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Floor Exercise,NaN
4,Nestor Abad,M,28,Spain,ESP,2020,Summer,Tokyo,Artistic Gymnastics,Men's Pommel Horse,NaN


In [35]:
olympics_df.to_csv('archive//olympics_2020_data.csv', index=False)